In [ ]:
import os

ENDWITHS = 'Pipelines'
NOTEBOOK_DIR = os.getcwd()

if not NOTEBOOK_DIR.endswith(ENDWITHS):
    raise ValueError(f"Not in correct dir, expect end with {ENDWITHS}, but got {NOTEBOOK_DIR} instead")

# Define the base directory relative to the current notebook's location.
BASE_DIR = os.path.join(NOTEBOOK_DIR, '..', '..', '..')

In [ ]:
import sys
# Add the project's 'code' directory to the Python path to import custom modules.
sys.path.insert(0, os.path.join(BASE_DIR, 'code'))

# Import necessary libraries and modules
from ultralytics import YOLO
from pipeline.SegmentationModels.YoloSeg import YoloSeg, plot_patch, plot_image
from pipeline.OCRModels.MangaOCRModel import MangaOCRModel
from pipeline.TranslationModels.ElanMtJaEnTranslator import ElanMtJaEnTranslator
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from math import ceil, floor
from PIL import Image, ImageDraw, ImageFont
import matplotlib.font_manager as fm

In [ ]:
# Define paths to the trained YOLO model and the example manga image.
YOLO_MODEL_PATH = os.path.join(BASE_DIR, 'best.pt')
EX_IMG_PATH = os.path.join(BASE_DIR, "data/Manga109_released_2023_12_07/images/AisazuNihaIrarenai/007.jpg")

In [ ]:
# Load the pre-trained YOLO model.
yolo_model = YOLO(YOLO_MODEL_PATH)

# Run the prediction on the source image.
results = yolo_model.predict(source=EX_IMG_PATH)

In [ ]:
# The prediction returns a list of results; 
# we are processing a single image, so we take the first element.
result = results[0]

In [ ]:
# Convert the original image from BGR (OpenCV's default) to RGB for display.
image_rgb = cv2.cvtColor(result.orig_img, cv2.COLOR_BGR2RGB)

# Extract the raw detection data from the YOLO result object.
boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
masks_xy = result.masks.xy              # Segmentation masks as polygon points

print(f"Initial YOLO detection found {len(boxes)} regions.")

In [ ]:
def split_connected_bubbles(bubble_mask):
    """
    Phiên bản nâng cao:
    1. Tìm các điểm khuyết (convexity defects).
    2. Lọc các điểm dựa trên độ sâu (depth) và góc nhọn (angle).
    3. Gom nhóm các điểm quá gần nhau (clustering).
    4. CHIẾN THUẬT CẮT:
       - Nếu tìm được >= 2 điểm khuyết mạnh: Nối 2 điểm sâu nhất lại với nhau (Cắt cổ chai).
       - Nếu chỉ tìm được 1 điểm: Cắt từ điểm đó về tâm (Centroid) như cũ.
    """
    
    # --- HYPERPARAMETERS ---
    MIN_DEFECT_DEPTH = 10   # Độ sâu tối thiểu của vết lõm (pixel)
    MAX_ANGLE_DEG = 130     # Góc tối đa (dưới 90 là nhọn, nhưng nới lên 100-110 cho an toàn)
    MIN_DIST_BETWEEN_DEFECTS = 20 # Khoảng cách tối thiểu để coi là 2 vết lõm khác nhau
    
    # Step 1: Tìm contour lớn nhất
    contours, _ = cv2.findContours(bubble_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return [bubble_mask]
    
    contour = max(contours, key=cv2.contourArea)
    if len(contour) < 10: # Contour quá nhỏ
        return [bubble_mask]

    # Step 2: Tính Convex Hull và Defects
    try:
        hull_indices = cv2.convexHull(contour, returnPoints=False)
        # ConvexityDefects yêu cầu contour và hull phải đúng chuẩn
        if hull_indices is None or len(hull_indices) < 3:
            return [bubble_mask]
            
        defects = cv2.convexityDefects(contour, hull_indices)
    except cv2.error:
        return [bubble_mask]

    if defects is None:
        return [bubble_mask]

    # Step 3: Tìm và Lọc các ứng viên (Candidates)
    candidates = [] # Lưu cấu trúc: {'point': (x,y), 'depth': float}
    
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        depth = d / 256.0 # OpenCV trả về depth * 256
        
        if depth > MIN_DEFECT_DEPTH:
            start_point = tuple(contour[s][0])
            end_point = tuple(contour[e][0])
            far_point = tuple(contour[f][0]) # Điểm lõm sâu nhất
            
            # Tính góc tại điểm far_point (dùng định lý cosin)
            # Vector từ far_point đến start và end
            v1 = np.array(start_point) - np.array(far_point)
            v2 = np.array(end_point) - np.array(far_point)
            
            norm1 = np.linalg.norm(v1)
            norm2 = np.linalg.norm(v2)
            
            if norm1 == 0 or norm2 == 0: continue
            
            # Cosine similarity
            cosine_angle = np.dot(v1, v2) / (norm1 * norm2)
            # Clip để tránh lỗi floating point
            angle_rad = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
            angle_deg = np.degrees(angle_rad)
            
            if angle_deg < MAX_ANGLE_DEG:
                candidates.append({'point': far_point, 'depth': depth})

    # Step 4: Gom nhóm các điểm quá gần nhau (Clustering)
    # Vì một vết lõm có thể sinh ra nhiều điểm defect sát nhau -> Lấy điểm sâu nhất
    candidates.sort(key=lambda x: x['depth'], reverse=True) # Sắp xếp sâu nhất lên đầu
    
    unique_candidates = []
    for cand in candidates:
        is_distinct = True
        for exist in unique_candidates:
            # Tính khoảng cách giữa điểm đang xét và điểm đã lưu
            dist = np.linalg.norm(np.array(cand['point']) - np.array(exist['point']))
            if dist < MIN_DIST_BETWEEN_DEFECTS:
                is_distinct = False
                break
        if is_distinct:
            unique_candidates.append(cand)

    # Step 5: Thực hiện cắt (Split Logic)
    split_mask = bubble_mask.copy()
    cut_performed = False

    if len(unique_candidates) >= 2:
        # --- TRƯỜNG HỢP 1: CÓ 2 ĐIỂM LÕM ĐỐI DIỆN (Như ảnh của bạn) ---
        # Lấy 2 điểm sâu nhất nối lại với nhau
        p1 = unique_candidates[0]['point']
        p2 = unique_candidates[1]['point']
        
        # Vẽ đường màu đen (cắt) nối 2 điểm này
        # Thickness = 2 hoặc 3 để đảm bảo tách rời hẳn
        cv2.line(split_mask, p1, p2, color=0, thickness=3)
        cut_performed = True
        
    elif len(unique_candidates) == 1:
        # --- TRƯỜNG HỢP 2: CHỈ CÓ 1 ĐIỂM LÕM (Bong bóng dính kiểu chữ V hoặc dấu phẩy) ---
        # Cắt từ điểm lõm về tâm (Centroid) của contour
        farthest_point = unique_candidates[0]['point']
        
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            # Kéo dài đường cắt qua tâm một chút để đảm bảo đứt hẳn
            dx = cx - farthest_point[0]
            dy = cy - farthest_point[1]
            # Điểm đích vượt quá tâm một chút
            target_x = int(cx + dx * 0.5)
            target_y = int(cy + dy * 0.5)
            
            cv2.line(split_mask, farthest_point, (target_x, target_y), color=0, thickness=3)
            cut_performed = True

    # Step 6: Trả về kết quả
    if cut_performed:
        # Tìm contour lại trên mask đã bị cắt
        new_contours, _ = cv2.findContours(split_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(new_contours) > 1:
            new_masks = []
            for c in new_contours:
                # Lọc bỏ nhiễu quá nhỏ sau khi cắt
                if cv2.contourArea(c) > 50: 
                    mask = np.zeros_like(bubble_mask)
                    cv2.drawContours(mask, [c], -1, 255, thickness=cv2.FILLED)
                    new_masks.append(mask)
            
            # Nếu cắt xong mà ra được >1 bong bóng hợp lệ thì trả về
            if len(new_masks) > 1:
                return new_masks

    # Nếu không cắt được hoặc cắt xong vẫn dính (do line quá mỏng), trả về mask gốc
    return [bubble_mask]

In [ ]:
# --- CELL MỚI: CÔNG CỤ TINH CHỈNH HYPERPARAMETER (INTERACTIVE) ---
# Chạy cell này để bật giao diện kéo thả.
# Sau khi tìm được 3 số ưng ý, hãy quay lại Cell 7 sửa vào code chính thức.

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# 1. Hàm Visualizer (Chỉ dùng để hiển thị debug, không ảnh hưởng logic chính)
def visualize_bubble_split(bubble_mask, min_depth, max_angle, min_dist):
    # Tạo ảnh màu để vẽ debug
    debug_img = cv2.cvtColor(bubble_mask, cv2.COLOR_GRAY2BGR)
    
    contours, _ = cv2.findContours(bubble_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return debug_img
    contour = max(contours, key=cv2.contourArea)
    
    cv2.drawContours(debug_img, [contour], -1, (0, 255, 255), 2) # Viền vàng

    try:
        hull_indices = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull_indices)
    except: return debug_img
    
    if defects is None: return debug_img

    candidates = []
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        depth = d / 256.0
        start = tuple(contour[s][0])
        end = tuple(contour[e][0])
        far = tuple(contour[f][0])

        v1 = np.array(start) - np.array(far)
        v2 = np.array(end) - np.array(far)
        norm1, norm2 = np.linalg.norm(v1), np.linalg.norm(v2)
        
        angle_deg = 180
        if norm1 > 0 and norm2 > 0:
            cosine = np.dot(v1, v2) / (norm1 * norm2)
            angle_deg = np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

        # Logic hiển thị: Xanh lá = Đạt, Đỏ = Trượt
        color = (0, 0, 255) 
        if depth > min_depth and angle_deg < max_angle:
            color = (0, 255, 0) 
            candidates.append({'point': far, 'depth': depth})
        
        cv2.circle(debug_img, far, 5, color, -1)
        # Ghi chú Depth và Angle cạnh điểm đó
        if depth > 10: # Chỉ hiện text cho điểm có độ sâu > 10 để đỡ rối
            cv2.putText(debug_img, f"{int(depth)}|{int(angle_deg)}", (far[0]+10, far[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Logic đường cắt
    candidates.sort(key=lambda x: x['depth'], reverse=True)
    unique_candidates = []
    for cand in candidates:
        is_distinct = True
        for exist in unique_candidates:
            if np.linalg.norm(np.array(cand['point']) - np.array(exist['point'])) < min_dist:
                is_distinct = False; break
        if is_distinct: unique_candidates.append(cand)

    if len(unique_candidates) >= 2:
        cv2.line(debug_img, unique_candidates[0]['point'], unique_candidates[1]['point'], (255, 0, 0), 3)
        cv2.putText(debug_img, "CUT: 2-POINTS", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    elif len(unique_candidates) == 1:
        p1 = unique_candidates[0]['point']
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx, cy = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])
            cv2.line(debug_img, p1, (cx, cy), (255, 0, 0), 3)
            cv2.putText(debug_img, "CUT: CENTROID", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    return debug_img

# 2. Hàm điều khiển chính
def interactive_tuner(bubble_index, min_depth, max_angle, min_dist):
    if len(masks_xy) == 0:
        print("Không tìm thấy mask nào từ YOLO!")
        return

    # Lấy mask từ kết quả YOLO (Cell 6)
    points = masks_xy[bubble_index]
    h, w = result.orig_img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(points, dtype=np.int32)], 255)

    # Crop (zoom) vào bong bóng đó để nhìn cho rõ
    x, y, w_b, h_b = cv2.boundingRect(np.array(points, dtype=np.int32))
    pad = 30
    y1, y2 = max(0, y-pad), min(h, y+h_b+pad)
    x1, x2 = max(0, x-pad), min(w, x+w_b+pad)
    mask_crop = mask[y1:y2, x1:x2]

    # Chạy visualizer
    viz_img = visualize_bubble_split(mask_crop, min_depth, max_angle, min_dist)

    # Vẽ hình
    plt.figure(figsize=(8, 8))
    plt.imshow(viz_img)
    plt.title(f"Bubble Index: {bubble_index} (Total: {len(masks_xy)})")
    plt.axis('off')
    plt.show()

# 3. Khởi tạo giao diện
print("--- CÔNG CỤ CHỈNH THAM SỐ ---")
print("1. Kéo 'bubble_index' để tìm bong bóng bị dính (hình số 8).")
print("2. Kéo 3 thanh còn lại để thấy đường cắt màu xanh dương (Blue) xuất hiện đúng chỗ.")
print("3. Ghi nhớ 3 số đó và điền lại vào Cell số 7.")

interact(interactive_tuner, 
         bubble_index=IntSlider(min=0, max=len(masks_xy)-1, step=1, value=0),
         min_depth=IntSlider(min=5, max=100, step=1, value=20, description='Depth'),
         max_angle=IntSlider(min=10, max=180, step=5, value=100, description='Angle'),
         min_dist=IntSlider(min=5, max=100, step=5, value=20, description='Dist'));

In [ ]:
# This list will store the final, individual bubbles after processing.
refined_bubble_list = []

print("Processing raw YOLO masks...")
# Iterate over each detected region from YOLO.
for mask_points in masks_xy:
    # Create a binary mask from the polygon points.
    initial_mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)
    cv2.fillPoly(initial_mask, [np.array(mask_points, dtype=np.int32)], 255)

    # Attempt to split the mask into individual bubbles.
    split_masks = split_connected_bubbles(initial_mask)

    # Process each resulting mask (could be one or more).
    for single_mask in split_masks:
        contours, _ = cv2.findContours(single_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue
        
        contour = max(contours, key=cv2.contourArea)
        # Calculate the precise bounding box for this individual bubble.
        bbox = cv2.boundingRect(contour)
        
        # Store the refined bubble information.
        refined_bubble_list.append({
            'mask': single_mask,
            'bbox': bbox,
            'contour': contour
        })

# Sort bubbles by reading order (top-to-bottom, then right-to-left for Japanese manga).
refined_bubble_list.sort(key=lambda b: (b['bbox'][1], -b['bbox'][0]))

print(f"After splitting, we have {len(refined_bubble_list)} individual bubbles.")

In [ ]:
# --- Create a Visualization of the Refined Bubbles ---

# Create a copy of the original image to draw on.
visualization_image = image_rgb.copy()
# Create another copy to use as an overlay for the transparent masks.
overlay = image_rgb.copy()

print("Generating visualization of the refined bubbles...")

# Set up the plot size based on the image's aspect ratio.
ratio = visualization_image.shape[1] / visualization_image.shape[0]
width = 20
height = width / ratio
fig, ax = plt.subplots(1, 1, figsize=(width, height))

# Iterate through the final list of individual bubbles.
for i, bubble in enumerate(refined_bubble_list):
    # --- 1. Draw the Segmentation Mask ---
    # Select a random color for each mask to distinguish them.
    color = np.random.randint(50, 255, size=3).tolist()
    contour = bubble['contour']
    # Draw the filled contour on the overlay image.
    cv2.drawContours(overlay, [contour], -1, color, thickness=cv2.FILLED)
    
    # --- 2. Draw the Bounding Box ---
    x, y, w, h = bubble['bbox']
    # Create a rectangle patch.
    rect = plt.Rectangle((x, y), w, h,
                         linewidth=2, edgecolor='lime', facecolor='none')
    ax.add_patch(rect)

    # --- 3. Add an ID Label to the Bounding Box ---
    ax.text(x, y - 5, f'ID: {i}',
            fontsize=15, color='black',
            bbox=dict(facecolor='lime', alpha=0.8, pad=0.5, edgecolor='none'),
            verticalalignment='bottom')

# --- Blend the Overlay with the Original Image ---
alpha = 0.4  # Transparency factor.
visualization_image = cv2.addWeighted(overlay, alpha, visualization_image, 1 - alpha, 0)

# --- Display the Final Plot ---
ax.imshow(visualization_image)
ax.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Initialize and load the OCR model.
manga_ocr_model = MangaOCRModel()
manga_ocr_model.load_model()

print("\nRunning OCR on each individual bubble...")
image_rgb_array = np.array(image_rgb)

# Iterate through the refined list of bubbles.
for i, bubble in enumerate(refined_bubble_list):
    x, y, w, h = bubble['bbox']
    
    # Crop the image using the precise bounding box.
    cropped_image = image_rgb_array[y:y+h, x:x+w]
    
    # Perform OCR on the cropped image.
    text = manga_ocr_model.predict(cropped_image)
    
    # Store the recognized text back into the bubble's dictionary.
    bubble['ocr_text'] = text
    
    print(f"Bubble ID {i}: {text}")

In [ ]:
# Initialize and load the translation model.
model_trans = ElanMtJaEnTranslator()
model_trans.load_model()

print("\n--- Translation Results ---")
# Iterate through the bubbles again to translate the OCR'd text.
for i, bubble in enumerate(refined_bubble_list):
    ocr_text = bubble.get('ocr_text', '')
    
    if ocr_text.strip():
        translated_text = model_trans.predict(ocr_text)
        bubble['translated_text'] = translated_text
    else:
        bubble['translated_text'] = ''

# Print the final results for verification.
for i, bubble in enumerate(refined_bubble_list):
    print(f"Bbox ID: {i}")
    print(f"  - OCR: {bubble.get('ocr_text', '')}")
    print(f"  - Translation: {bubble.get('translated_text', '')}\n")

In [ ]:
FONT_PATH = "/home/zendragonxxx/miniconda3/envs/myenv/lib/python3.12/site-packages/matplotlib/mpl-data/fonts/ttf/ComicSansMS3.ttf"
    
def generate_lines(draw, text, font, max_width):
    """
    Intelligently wraps text to fit within a maximum width.
    It can also break long words that exceed the max_width.
    """
    lines = []
    words = text.split()
    
    current_line = ""
    for word in words:
        # Handle the case where a single word is longer than the allowed width.
        if draw.textbbox((0,0), word, font=font)[2] > max_width:
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            
            temp_word = ""
            for char in word:
                if draw.textbbox((0,0), temp_word + char, font=font)[2] <= max_width:
                    temp_word += char
                else:
                    lines.append(temp_word)
                    temp_word = char
            current_line = temp_word
            continue

        # Standard word wrapping logic.
        if not current_line:
             current_line = word
        elif draw.textbbox((0,0), current_line + " " + word, font=font)[2] <= max_width:
            current_line += " " + word
        else:
            lines.append(current_line.strip())
            current_line = word
            
    if current_line:
        lines.append(current_line.strip())
        
    return lines

def fit_text_in_bubble(draw, text, font_path, bounding_box, text_color):
    """
    Finds the largest possible font size for the text to fit inside the
    bounding box, prioritizing line wrapping before reducing font size.
    """
    x, y, w, h = bounding_box
    if w <= 0 or h <= 0 or not font_path: return

    font_size = h
    final_lines = []
    final_font = None

    # Start with a large font size and decrease until the text fits.
    while font_size > 5:
        font = ImageFont.truetype(font_path, font_size)
        lines = generate_lines(draw, text, font, w)
        total_height = sum([draw.textbbox((0, 0), line, font=font)[3] for line in lines])
        
        if total_height <= h:
            final_lines = lines
            final_font = font
            break
        
        font_size -= 3
    
    # If no suitable size was found, use the last (smallest) tried size.
    if not final_font:
        final_font = ImageFont.truetype(font_path, font_size)
        final_lines = generate_lines(draw, text, final_font, w)

    # Draw the final text, centered within the bounding box.
    total_height = sum([draw.textbbox((0, 0), line, font=final_font)[3] for line in final_lines])
    current_y = y + (h - total_height) / 2
    
    for line in final_lines:
        line_height = draw.textbbox((0, 0), line, font=final_font)[3]
        line_width = draw.textbbox((0, 0), line, font=final_font)[2]
        current_x = x + (w - line_width) / 2
        draw.text((current_x, current_y), line, font=final_font, fill=text_color)
        current_y += line_height

In [ ]:
EROSION_PIXELS = 3
TEXT_COLOR = (0, 0, 0) # Black

image_final = image_rgb.copy()
erosion_kernel = np.ones((2 * EROSION_PIXELS + 1, 2 * EROSION_PIXELS + 1), np.uint8)

# Iterate through the final, complete list of individual bubbles.
for bubble in refined_bubble_list:
    trans_text = bubble.get('translated_text', '')
    if not trans_text.strip():
        continue

    single_mask = bubble['mask']

    # STEP 1: Erode the mask to create a safe margin from the original text border.
    eroded_mask = cv2.erode(single_mask, erosion_kernel, iterations=1)
    
    # STEP 2: Find the contours of the eroded mask and paint the area white to clear it.
    contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image_final, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    
    # STEP 3: Find the bounding box of the cleared area, which is where we will draw the text.
    if contours:
        text_area_bbox = cv2.boundingRect(contours[0])
        
        # STEP 4: Convert to PIL Image and call the text fitting function.
        pil_image = Image.fromarray(image_final)
        draw = ImageDraw.Draw(pil_image)
        
        fit_text_in_bubble(draw, trans_text, FONT_PATH, text_area_bbox, TEXT_COLOR)
        
        # Convert back to numpy array for the next iteration.
        image_final = np.array(pil_image)

# --- Display the final, typeset image ---
ratio = image_final.shape[1] / image_final.shape[0]
width = 20
height = width / ratio

fig, ax = plt.subplots(1, 1, figsize=(width, height))
ax.imshow(image_final)
ax.axis('off')
plt.tight_layout(pad=0)
plt.show()